In [1]:
import models
from models import model, create_cd_cluster, cd_diagram, bt_plot, glrt, model_builder
from dataset_management import get_dataset, convert_to_autorank
from autorank import autorank, create_report, plot_stats
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

algorithm="algorithm";benchmark="bench_prior";seed="seed";budget="used_fidelity";loss="regret"

In [2]:
data=get_dataset(dataset_name="fig5_24_at25_bad_intsteps")
for f in [[0,1],[2,3],[4,5],[6,7],[8,9],[10,11]]:
    dataf=data.loc[(data[budget].between(f[0],f[1]))&data[algorithm].isin(["HB","PB"])]
    print(f"Budget: {f[0]}-{f[1]}")
    builder=model_builder(dataf,"regret",algorithm,benchmark)
    builder.test_benchmark_information()


Budget: 0-1
Benchmark LC-167190_bad is informative.
Benchmark JAHS-C10_at25 is informative.
Benchmark LC-126026_bad is informative.
Benchmark PD1-Cifar100_at25 is informative.
Benchmark PD1-ImageNet_at25 is informative.
Benchmark LC-126026_at25 is informative.
Benchmark LC-167190_at25 is uninformative.
Benchmark LC-168330_bad is informative.
Benchmark JAHS-CH_at25 is informative.
Benchmark LC-168910_bad is informative.
Benchmark PD1-LM1B_bad is informative.
Benchmark PD1-WMT_bad is informative.
Benchmark PD1-WMT_at25 is informative.
Benchmark JAHS-FM_at25 is informative.
Benchmark JAHS-C10_bad is informative.
Benchmark LC-168910_at25 is informative.
Benchmark LC-189906_bad is informative.
Benchmark PD1-Cifar100_bad is informative.
Benchmark PD1-LM1B_at25 is informative.
Benchmark LC-189906_at25 is informative.
Benchmark JAHS-CH_bad is informative.
Benchmark JAHS-FM_bad is informative.
Benchmark PD1-ImageNet_bad is informative.
Benchmark LC-168330_at25 is uninformative.
Budget: 2-3
Benc

In [3]:
model_list={}
p_val_collection=pd.DataFrame(columns=["value_type","bench_prior","prior","fidelity","PB - RS","HB - PB","RS - HB"])
for fidelity in range(1,18):
    data_pb_hb=data.loc[(data[budget].between(fidelity,fidelity+0.5))]#(data[algorithm].isin(["HB","PB"]))&
    for bench in data[benchmark].unique():
        model_list[benchmark] = model(f"{loss}~{algorithm}+{budget}:{algorithm}", data_pb_hb.loc[data_pb_hb[benchmark]==bench])
        estimate,contrast=model_list[benchmark].post_hoc(marginal_vars=[algorithm])
        p_val=[0,0,0]
        for i,algos in enumerate(contrast["Contrast"].values):
            p_val[i]=contrast["P-val"].values[i]*np.sign(contrast["Estimate"].values[i])
        p_val_collection.loc[len(p_val_collection)]=[loss,bench,bench.rsplit("_",1)[1],f"{fidelity}-{fidelity+0.5}"]+p_val
    print(p_val_collection)

boundary (singular) fit: see help('isSingular') 

P-values adjusted by tukey method for family of 3 estimates
boundary (singular) fit: see help('isSingular') 

P-values adjusted by tukey method for family of 3 estimates
boundary (singular) fit: see help('isSingular') 

P-values adjusted by tukey method for family of 3 estimates
boundary (singular) fit: see help('isSingular') 

P-values adjusted by tukey method for family of 3 estimates
boundary (singular) fit: see help('isSingular') 

P-values adjusted by tukey method for family of 3 estimates
boundary (singular) fit: see help('isSingular') 

P-values adjusted by tukey method for family of 3 estimates
boundary (singular) fit: see help('isSingular') 

P-values adjusted by tukey method for family of 3 estimates
boundary (singular) fit: see help('isSingular') 

P-values adjusted by tukey method for family of 3 estimates
boundary (singular) fit: see help('isSingular') 

P-values adjusted by tukey method for family of 3 estimates
boundary (

In [4]:
%matplotlib ipympl
def simple_clustering(k,data_frame, type_column, numerical_columns, value,fidelity,plot:bool=False):
    # Combine data from all categories
    combined_data = pd.DataFrame()
    for type_value in data_frame[type_column].unique():
        type_data = data_frame[data_frame[type_column] == type_value]
        combined_data = pd.concat([combined_data,type_data])

    # Perform clustering on the combined data
    combined_data=data_frame
    kmeans = KMeans(n_clusters=k,n_init="auto")
    kmeans.fit(combined_data[numerical_columns])
    cluster_labels = kmeans.labels_
    

    # Assign cluster labels to the original data frame
    data_frame['cluster'] = cluster_labels
    def add_cluster_centers(row,columns,means):
        return means[row["cluster"]]

    # data_frame[numerical_columns] = data_frame.apply(add_cluster_centers, axis=1,columns=numerical_columns,means=kmeans.cluster_centers_).to_list()
    # print(combined_data[numerical_columns])

    if plot:
        # Visualize clusters using Matplotlib
        fig=plt.figure()
        ax=fig.add_subplot(projection="3d")
        
        ax.scatter(combined_data[numerical_columns[0]], combined_data[numerical_columns[1]],combined_data[numerical_columns[2]], c=cluster_labels)
        
        for _,row in combined_data.iterrows():
            ax.text(row[numerical_columns].values[0],row[numerical_columns].values[1],row[numerical_columns].values[2],row["prior"],fontsize=5)

        
        plt.title(f"{k} Clusters in {value} and fidelity {fidelity}")
        # ax.set_xlim([-1, 1])
        # ax.set_ylim([-1, 1])
        # ax.set_zlim([-1,1])
        ax.set_xlabel(numerical_columns[0])
        ax.set_ylabel(numerical_columns[1])
        ax.set_zlabel(numerical_columns[2])
        plt.show()

    return data_frame

clusters={}
for value in p_val_collection["value_type"].unique():
    clusters[value]={}
    for fidelity in p_val_collection["fidelity"].unique():
        sample_df=p_val_collection.loc[(p_val_collection["value_type"]==value)&(p_val_collection["fidelity"]==fidelity)][["bench_prior","PB - RS","HB - PB","RS - HB","prior"]]
        clusters[value][fidelity]=simple_clustering(2,sample_df,benchmark,["HB - PB"],value,fidelity,plot=False)

In [5]:
for value in p_val_collection["value_type"].unique():
    for fidelity in p_val_collection["fidelity"].unique():
        print(clusters[value][fidelity].sort_values(by=["cluster","prior"]))

          bench_prior  PB - RS  HB - PB  RS - HB prior  cluster
1       JAHS-C10_at25     -0.0   -0.000   -0.000  at25        0
3   PD1-Cifar100_at25     -0.0   -0.000    0.053  at25        0
4   PD1-ImageNet_at25     -0.0   -0.000    0.000  at25        0
5      LC-126026_at25     -0.0   -0.065    0.008  at25        0
8        JAHS-CH_at25     -0.0   -0.000   -0.000  at25        0
12       PD1-WMT_at25     -0.0   -0.000    0.000  at25        0
13       JAHS-FM_at25     -0.0   -0.000   -0.000  at25        0
15     LC-168910_at25     -0.0   -0.141    0.000  at25        0
18      PD1-LM1B_at25     -0.0   -0.001    0.000  at25        0
19     LC-189906_at25     -0.0   -0.013    0.000  at25        0
0       LC-167190_bad      0.0    0.000    0.000   bad        0
2       LC-126026_bad      0.0    0.000    0.008   bad        0
7       LC-168330_bad      0.0    0.000    0.000   bad        0
9       LC-168910_bad      0.0    0.000    0.000   bad        0
10       PD1-LM1B_bad      0.0    0.000 

In [7]:

# print(data.bench_prior.unique())

# # Extend dataset by metafeatures
# mfs=pd.read_csv("benchmark_metafeatures.csv").drop(["Name","log_int_fid","n_cond_Vars"],axis=1)
# mf_names=list(mfs.columns[1:])
# def add_mf(row):
#     return pd.Series(mfs.loc[mfs["benchmark"]==row[benchmark].rsplit("_",1)[0]].values[0][1:])
# data[["n_Vars","n_cont_Vars","n_cat_Vars","int_fid"]]=data.apply(add_mf,axis=1)


# # Build the model

# # Base-model
# base_model = model(f"{loss}~{algorithm}", data)
# print(base_model)

# # Extend the model by testing which metafeatures has the biggest impact
# model_list={}
# for mf in mf_names:
#     print(mf)
#     model_list[mf] = model(f"{loss}~{algorithm}+{mf}", data)


# for mf in mf_names:
#     glrt(model_list[mf], base_model,names=[mf, "base"])


ranked_models = sorted(list(model_list.items())+[("Base model",base_model)], key=lambda x: x[1].logLike, reverse=True)

for model_name, model_obj in ranked_models:
    print(f"Model: {model_name}")
    print(f"LogLike: {model_obj.logLike}")


boundary (singular) fit: see help('isSingular') 

pymer4.models.Lmer(fitted = True, formula = regret~algorithm+(1|dummy), family = gaussian)
n_Vars
boundary (singular) fit: see help('isSingular') 

n_cont_Vars
boundary (singular) fit: see help('isSingular') 

n_cat_Vars
boundary (singular) fit: see help('isSingular') 

int_fid
boundary (singular) fit: see help('isSingular') 

n_Vars (54576.16) >> base (49198.07)
Chi-Square: 10756.163055213168, P-Value: 0.0
n_cont_Vars (55569.55) >> base (49198.07)
Chi-Square: 12742.950858559212, P-Value: 0.0
n_cat_Vars (55569.55) >> base (49198.07)
Chi-Square: 12742.950858559372, P-Value: 0.0
int_fid (49496.79) >> base (49198.07)
Chi-Square: 597.4383647486102, P-Value: 0.0
Model: n_cat_Vars
LogLike: 55569.55009105122
Model: n_cont_Vars
LogLike: 55569.55009105114
Model: n_Vars
LogLike: 54576.15618937812
Model: int_fid
LogLike: 49496.79384414584
Model: Base model
LogLike: 49198.074661771534
